In [ ]:
pip install cvxopt

In [ ]:
from cvxopt import matrix
from cvxopt import solvers
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
(a,b,c) = (1,1,1) # ax+by+c = 0
Npoints = 10000
offset_negative = 5+10*np.random.random((Npoints//2,1))
offset_positive = 5+10*np.random.random((Npoints//2,1))
x_negative = -20*np.random.random((Npoints//2,1)) # Negative class
x_positive = 20*np.random.random((Npoints//2,1))  # Positive class
y_negative = -(a/b)*x_negative-(c/b) - offset_negative
y_positive = -(a/b)*x_positive-(c/b) + offset_positive
labels_negative = -1*np.ones((Npoints//2,1))
labels_positive = np.ones((Npoints//2,1))

x_all = np.r_[x_negative,x_positive]
y_all = np.r_[y_negative,y_positive]
y_all_line = -(a/b)*x_all-(c/b)
labels_all = np.r_[labels_negative,labels_positive]

X = np.c_[x_all,y_all,np.ones((Npoints,1))]
print(f'X.shape: {X.shape}')

Diag = np.zeros((Npoints,Npoints))
np.fill_diagonal(Diag, labels_all)
print(f'Diag.shape: {Diag.shape}')

G_array = np.dot(Diag,X)
print(f'G.shape: {G_array.shape}')

In [ ]:
plt.plot(x_all,y_all_line)
plt.scatter(x_all,y_all)
plt.grid()
plt.show()

In [ ]:
P_array = np.array([[1,0,0],[0,1,0],[0,0,0]])
P = matrix(P_array,tc='d')
G = matrix(G_array,tc='d')
q = matrix(np.array([0,0,0]),tc='d')
h = matrix(-1*np.ones((Npoints,1)), tc='d')

In [ ]:
sol = solvers.qp(P,q,G,h)

In [ ]:
len(sol['x'])

In [ ]:
print(sol['x'][0],sol['x'][1],sol['x'][2])
print(np.square(sol['x'][0])+np.square(sol['x'][1]+np.square(sol['x'][2])))

In [ ]:
y_all_line_after = -(sol['x'][0]/sol['x'][1])*x_all-(sol['x'][2]/sol['x'][1])
plt.plot(x_all,y_all_line_after)
plt.scatter(x_all,y_all)
plt.grid()
plt.show()

In [ ]:
plt.plot(x_all,y_all_line)
plt.scatter(x_all,y_all)
plt.grid()
plt.show()

In [ ]:
import tensorflow as tf

In [ ]:
LogRegModel = tf.keras.Sequential()
LogRegModel.add(tf.keras.layers.Input(shape=(Npoints,)))
LogRegModel.add(tf.keras.layers.Dense(1, activation='sigmoid'))
LogRegModel.output_shape

In [ ]:
X_Train = np.c_[x_all,y_all]
y_Train = np.r_[labels_negative,labels_positive]
print(f'X_Train.shape: {X_Train.shape}')
print(f'y_Train.shape: {y_Train.shape}')

Npoints_test = 5000
offset_negative = 5+10*np.random.random((Npoints_test//2,1))
offset_positive = 5+10*np.random.random((Npoints_test//2,1))
x_negative_test = -20*np.random.random((Npoints_test//2,1)) # Negative class
x_positive_test = 20*np.random.random((Npoints_test//2,1))  # Positive class
y_negative_test = -(a/b)*x_negative_test-(c/b) - offset_negative
y_positive_test = -(a/b)*x_positive_test-(c/b) + offset_positive
labels_negative_test = -1*np.ones((Npoints_test//2,1))
labels_positive_test = np.ones((Npoints_test//2,1))

x_all_test = np.r_[x_negative_test,x_positive_test]
y_all_test = np.r_[y_negative_test,y_positive_test]
X_Test = np.c_[x_all_test,y_all_test]
y_Test = np.r_[labels_negative_test,labels_positive_test]
print(f'X_Test.shape: {X_Test.shape}')
print(f'y_Test.shape: {y_Test.shape}')



### WITH INPUT LAYER

In [ ]:
model1 = tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape=(2,)),
  tf.keras.layers.Dense(1,activation='sigmoid')])
model1.compile(optimizer='sgd', loss=tf.keras.losses.BinaryCrossentropy())
model1.fit(X_Train,y_Train)

### WIHTOUT INPUT LAYER

In [ ]:
model1.predict(X_Test)

In [ ]:
model2 = tf.keras.Sequential([  
  tf.keras.layers.Dense(1,input_shape=(2,),activation='sigmoid')])
model2.compile(optimizer='sgd', loss=tf.keras.losses.BinaryCrossentropy())
model2.fit(X_Train,y_Train)

In [ ]:
model2.predict(X_Test)

In [ ]:
model2.summary()

In [ ]:
wts1 = model1.get_weights()
wts2 = model2.get_weights()

In [ ]:
a_hat1 = wts1[0][0]
b_hat1 = wts1[0][1]
c_hat1 = wts1[1]

a_hat2 = wts2[0][0]
b_hat2 = wts2[0][1]
c_hat2 = wts2[1]

In [ ]:
y_all_line_NN1 = -(a_hat1/b_hat1)*x_all-(c_hat1/b_hat1)
plt.plot(x_all,y_all_line_NN1)
plt.scatter(x_all,y_all)
plt.grid()
plt.show()

In [ ]:
y_all_line_NN2 = -(a_hat2/b_hat2)*x_all-(c_hat2/b_hat2)
plt.plot(x_all,y_all_line_NN2)
plt.scatter(x_all,y_all)
plt.grid()
plt.show()

In [ ]:
print(a_hat1,b_hat1,c_hat1)
print(a_hat2,b_hat2,c_hat2)

In [ ]:
plt.plot(x_all,y_all_line_NN1,'-b')
plt.plot(x_all,y_all_line_after,'-k')
plt.legend(['LogReg','SVM'])
plt.grid()
plt.show()